In [11]:
from keras.models import Sequential
from keras.layers import Dense, Dropout
from keras.layers import LSTM
from sklearn.model_selection import train_test_split
import numpy as np
import matplotlib.pyplot as plt
import tensorflow as tf

In [168]:
# confirm TensorFlow sees the GPU
from tensorflow.python.client import device_lib
assert 'GPU' in str(device_lib.list_local_devices())

# confirm Keras sees the GPU
from keras import backend
assert len(backend.tensorflow_backend._get_available_gpus()) > 0

In [49]:
def normalize_data(x_data, y_data):
    # Normalize data according to max/min range
    # x format: (d0, z0, phi, theta, q/p, time, vertex ID)
    # y format: (vx, vy, vz)
    # d0 unchanged, z0 -> z0 /200, phi -> phi/PI, theta -> theta/PI, qp-> qp/3
    # Omit time and vtx idx for now
    
    # Define normalization factors
    normZ0 = 200
    normPhi = np.pi
    normTheta = np.pi
    normQP = 3
    normVx = 3
    normVy = 3
    normVz = 200
    
    for event in x_data:
        for i,trk in enumerate(event):
            trk[1]/=normZ0
            trk[2]/=normPhi
            trk[3]/=normTheta
            trk[4]/=normQP

    # vx -> vx/3, vy->vy/3, vz -> vz/200       
    for event in y_data:
        event[0][0]/=normVx
        event[0][1]/=normVy
        event[0][2]/=normVz
        
def flattenAndPad(data):
    new_x = []
    for x in data:
        zeros = np.zeros(600)
        flat = x.flatten()
        zeros[:flat.shape[0]] = flat
        new_x.append(zeros)

    return np.array(new_x)

In [132]:
# Load and prepare data
array_reloaded = np.load("VertexAndTracksCollection_n20000_p0.npz")
tracks = array_reloaded["trackCollection"]
vertices = array_reloaded["vertexCollection"]

# Normalize data
normalize_data(tracks, vertices)
#print(data[0][0])

x_train, x_test, y_train, y_test = train_test_split(tracks, vertices,test_size=0.2, random_state=4)

x_train = flattenAndPad(x_train)
x_test = flattenAndPad(x_test)